# Prediction of Future Closing Stock Prices with Machine Learning v1.2

## Preliminaries: Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Part 1.1: Predict JFC stocks from 3/5/18 up to the present day then compare with latest      WSJ Data 

Since Google Finance data is not currently updated, we could get data from Wall Street Journal. Instead of using an API, we have to download csv files from the website.
Download csv file of JFC prices from 2013-1-1 up to the latest.

Source: http://quotes.wsj.com/PH/XPHS/JFC/historical-prices

Import data as a Pandas dataframe.

In [2]:
stock = pd.read_csv('JFC_WSJ.csv')

Inspect dataframe.

In [3]:
stock.columns

Index(['Date', ' Open', ' High', ' Low', ' Close', ' Volume'], dtype='object')

## 1.2: Data Cleaning 

Remove spaces from column names by renaming them.

In [4]:
stock.rename(columns=lambda x: x.strip(), inplace=True)
stock.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

Inspect the data types of each column, then the first 5 and last 5 rows of the dataframe.

In [5]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276 entries, 0 to 1275
Data columns (total 6 columns):
Date      1276 non-null object
Open      1276 non-null float64
High      1276 non-null float64
Low       1276 non-null float64
Close     1276 non-null float64
Volume    1276 non-null int64
dtypes: float64(4), int64(1), object(1)
memory usage: 59.9+ KB


In [6]:
stock.head()

,Date,Open,High,Low,Close,Volume
0,03/28/18,295.0,299.0,292.0,299.0,759390
1,03/27/18,295.0,297.0,289.4,294.4,356420
2,03/26/18,286.6,295.0,282.0,295.0,404650
3,03/23/18,285.0,288.0,280.0,288.0,864870
4,03/22/18,285.4,288.0,282.4,286.8,573890


In [7]:
stock.tail()

,Date,Open,High,Low,Close,Volume
1271,01/08/13,107.1,107.7,107.1,107.3,389530
1272,01/07/13,107.5,107.5,106.7,107.0,719780
1273,01/04/13,107.0,107.6,107.0,107.6,548040
1274,01/03/13,104.8,107.5,104.8,107.2,684290
1275,01/02/13,102.0,104.5,102.0,104.2,125900


Find nan values. Replace them with -999999 if there are present.

In [8]:
stock.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

No NaN values!!!
However, we have to rearrange the data from descending to ascending order.

In [9]:
stock = stock.reindex(index=stock.index[::-1]).reset_index(drop=True)

In [10]:
stock.head()

,Date,Open,High,Low,Close,Volume
0,01/02/13,102.0,104.5,102.0,104.2,125900
1,01/03/13,104.8,107.5,104.8,107.2,684290
2,01/04/13,107.0,107.6,107.0,107.6,548040
3,01/07/13,107.5,107.5,106.7,107.0,719780
4,01/08/13,107.1,107.7,107.1,107.3,389530


In [11]:
stock.tail()

,Date,Open,High,Low,Close,Volume
1271,03/22/18,285.4,288.0,282.4,286.8,573890
1272,03/23/18,285.0,288.0,280.0,288.0,864870
1273,03/26/18,286.6,295.0,282.0,295.0,404650
1274,03/27/18,295.0,297.0,289.4,294.4,356420
1275,03/28/18,295.0,299.0,292.0,299.0,759390


Convert 'Date' column as datetime objects and set it as index column.

In [12]:
stock['Date'] = pd.to_datetime(stock['Date'], infer_datetime_format=True)

In [13]:
stock.set_index('Date',inplace=True)

In [14]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1276 entries, 2013-01-02 to 2018-03-28
Data columns (total 5 columns):
Open      1276 non-null float64
High      1276 non-null float64
Low       1276 non-null float64
Close     1276 non-null float64
Volume    1276 non-null int64
dtypes: float64(4), int64(1)
memory usage: 59.8 KB


In [15]:
type(stock.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [16]:
stock.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2018-03-22,285.4,288.0,282.4,286.8,573890
2018-03-23,285.0,288.0,280.0,288.0,864870
2018-03-26,286.6,295.0,282.0,295.0,404650
2018-03-27,295.0,297.0,289.4,294.4,356420
2018-03-28,295.0,299.0,292.0,299.0,759390


In [17]:
stock['HL_PCT'] = (stock['High'] - stock['Close']) / stock['Close'] * 100
stock['PCT_change'] = (stock['High']-stock['Open']) / stock['Open'] * 100

In [18]:
stock.tail()

,Open,High,Low,Close,Volume,HL_PCT,PCT_change
Date,,,,,,,
2018-03-22,285.4,288.0,282.4,286.8,573890,0.418410,0.911002
2018-03-23,285.0,288.0,280.0,288.0,864870,0.000000,1.052632
2018-03-26,286.6,295.0,282.0,295.0,404650,0.000000,2.930914
2018-03-27,295.0,297.0,289.4,294.4,356420,0.883152,0.677966
2018-03-28,295.0,299.0,292.0,299.0,759390,0.000000,1.355932


Create Copy for exercise.

In [19]:
stockC = stock.copy(deep=True)
stockC.tail(10)

,Open,High,Low,Close,Volume,HL_PCT,PCT_change
Date,,,,,,,
2018-03-15,288.4,288.6,282.6,284.0,1140780,1.619718,0.069348
2018-03-16,286.8,305.4,286.8,305.4,2052470,0.000000,6.485356
2018-03-19,299.6,299.6,295.2,298.0,531280,0.536913,0.000000
2018-03-20,296.8,296.8,285.0,286.0,1028940,3.776224,0.000000
2018-03-21,286.6,293.0,285.0,285.0,953100,2.807018,2.233077
2018-03-22,285.4,288.0,282.4,286.8,573890,0.418410,0.911002
2018-03-23,285.0,288.0,280.0,288.0,864870,0.000000,1.052632
2018-03-26,286.6,295.0,282.0,295.0,404650,0.000000,2.930914
2018-03-27,295.0,297.0,289.4,294.4,356420,0.883152,0.677966


Notice that there are no values for 03-03 and 03-04. Trading days exclude weekends and holidays.

Add Prediction column which contains the predicted Close column value n trading days later.

We'll try to predict the 'Close' values from 03-05 up to the latest day available.

In [20]:
day_start = stockC.index.get_loc(pd.Timestamp('2018-03-05'))
day_start

1258

In [21]:
day_end = len(stockC.index)
day_end

1276

In [22]:
n = day_end-day_start
n

18

We'll add a 'Prediction column' that contains the actual closing values n days later.

In [23]:
stockC['Prediction'] = stockC['Close'].shift(-n)
stockC.tail(n+1)

,Open,High,Low,Close,Volume,HL_PCT,PCT_change,Prediction
Date,,,,,,,,
2018-03-02,297.6,298.4,295.0,298.0,735080,0.134228,0.268817,299.0
2018-03-05,299.0,300.0,296.0,299.6,388840,0.133511,0.334448,NaN
2018-03-06,300.0,304.0,297.8,298.6,833460,1.808439,1.333333,NaN
2018-03-07,300.0,304.0,299.8,304.0,853340,0.000000,1.333333,NaN
2018-03-08,303.0,304.0,295.0,295.0,367970,3.050847,0.330033,NaN
2018-03-09,297.0,297.0,292.0,292.0,352590,1.712329,0.000000,NaN
2018-03-12,296.0,299.0,292.2,295.4,303520,1.218687,1.013514,NaN
2018-03-13,296.0,297.0,292.4,293.0,557650,1.365188,0.337838,NaN
2018-03-14,293.0,294.4,287.4,288.4,757340,2.080444,0.477816,NaN


Notice that...

The predicted value in 2018-03-02, 299.0,

In [24]:
stockC['Prediction'].index[-n-1]

Timestamp('2018-03-02 00:00:00')

In [25]:
stockC['Prediction'][stockC['Prediction'].index[-n-1]]

299.0

Is the same as the closing price in 2018-03-28.

In [26]:
stockC['Close'].index[-1]

Timestamp('2018-03-28 00:00:00')

In [27]:
stockC['Close'][stockC['Close'].index[-1]]

299.0

## 1.3: Preprocessing 

The dataset would be divided as X and y; with X as... and y as...

In order to prepare the data for machine learning, we have to preprocess the X in order to...

In [28]:
from sklearn import preprocessing

In [29]:
X = preprocessing.scale(stockC.drop(['Prediction'], axis=1))

X would be split into two..;X would be the data before...

X_predict would be the data after... and would be used later to predict the future closing values.

In [30]:
X_predict = X[-n:]
X = X[:-n]

In [31]:
y = stockC['Prediction'][:-n]

In [32]:
print(type(X), type(X_predict), type(y))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'pandas.core.series.Series'>


In [33]:
print(len(X), len(X_predict), len(y))

1258 18 1258


## 1.4:  Cross Validation

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

clf = SVR(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=5)
scores #score for each split

array([0.84060057, 0.78868771, 0.08058599, 0.56425953, 0.75625778])

In [36]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.61 (+/- 0.56)


In [37]:
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('accuracy is %f' %accuracy)

NameError: name 'X_train' is not defined

## 1.5: Train Test Split 

Both X and y would be split into two; train set and test set. 

Train set would be used to train the computer while the test set would be used to compare and evaluate the predictions of the computer.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(0.2))

In [ ]:
print(type(X_train), type(X_test), type(y_train), type(y_test) )

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test) )

## 1.6: Try with Linear Regression 

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
accuracy = lm.score(X_test, y_test)
print('Accuracy: '+str(accuracy*100)+'%')

## 1.7: Try with all regressors

In [ ]:
from sklearn import preprocessing#, cross_validation
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor as NNR
from sklearn.linear_model import LinearRegression as LRR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.gaussian_process import GaussianProcessRegressor as GPR

predictors = [SVR(), NNR(), LRR(), RFR(), DTR(), GPR()]

forecast = []
pred_cols = []
accuracy = []

fig, ax = plt.subplots(6,2, figsize=(12,24))

for pred, color, i in zip(predictors, sns.color_palette(), range(len(predictors))):
    pred.fit(X_train, y_train)
    accu = pred.score(X_test,y_test)
    predictions = pred.predict(X_test)
            
    sns.regplot(y_test,predictions, color=color, ax=ax[i,0])
    ax[i,0].set_title('{0} accuracy = {1:.2f}'.format(str(pred)[:3], (accu*100).round(2))+'%')
    ax[i,0].set_xlabel('Y Test')
    ax[i,0].set_ylabel('Predicted Y')
    
    sns.distplot((y_test-predictions), bins=50, color=color, ax=ax[i,1])
    ax[i,1].set_title('{0} Residual Histogram'.format(str(pred)[:3]))
    
    forecast.append(pred.predict(X_predict))
    pred_cols.append(str(pred)[:3])
    accuracy.append('{0} accuracy = {1:.2f}'.format(str(pred)[:3], (accu*100).round(2))+'%')
    
fig.suptitle('Performance of Scikit-learn Regressors for Closing Stock Price Prediction', fontsize=15, y=1.01)
sns.set_style('whitegrid')
fig.tight_layout()

##  1.8: Add forecast columns per regressor

Forecast: Data for columns

In [ ]:
np.array(forecast).shape

In [ ]:
np.array(forecast)

In [ ]:
forecast_T = np.array(forecast).transpose()
forecast_T

In [ ]:
type(forecast_T)

In [ ]:
forecast_T.shape

Days: Index

In [ ]:
len(stockC.index)

In [ ]:
stockC.index[-1]

In [ ]:
Days=stockC.index[-n:len(stockC.index)]
Days

In [ ]:
range(len(stockC.index[-n:len(stockC.index)]))

Accuracy: column names

In [ ]:
type(accuracy)

In [ ]:
accuracy

In [ ]:
regs_stockC = pd.DataFrame(data=forecast_T,
                    index=stockC.index[-n:len(stockC.index)],
                    columns=accuracy)
regs_stockC

In [ ]:
accuracy

Actual Closing Price the trading day before the forecast.

In [ ]:
print(stockC.index[-n-1])
print(stockC['Close'][-n-1])

In [ ]:
rowlink = []
for i in regs_stockC.columns:
    rowlink.append(stockC['Close'][-n-1])
np.array([rowlink])

In [ ]:
range(len(stockC.index[-n:len(stockC.index)]))

In [ ]:
stockC.index[-n-1:len(stockC.index)]

In [ ]:
accuracy

In [ ]:
stockC.index[-n-1:-n]

In [ ]:
link = pd.DataFrame(data=np.array([rowlink]), 
                    index=stockC.index[-n-1:-n],
                    columns=accuracy)
link

# Merge the dataframes

In [ ]:
predic_stockC = pd.merge(link,regs_stockC, how='outer').set_index(stockC.index[-n-1:len(stockC.index)])
predic_stockC

In [ ]:
from IPython.display import YouTubeVideo
from datetime import timedelta

start=int(timedelta(seconds=10).total_seconds())
end=int(timedelta(seconds=14).total_seconds())

YouTubeVideo("VA41XQMUi-4", start=start, end=end, autoplay=1)

## 1.9.1: Plot all Regressor Predictions 

In [ ]:
label='JFC.PH'

In [ ]:
ax = stockC['Close'].plot(color='black', label='Actual Values', figsize=(10,5), lw=2,marker='o', markersize=8)
predic_stockC.plot(color=sns.color_palette(),marker='o', ax=ax)

plt.title('{0} Stocks from {1} to {2}'.format(label,str(predic_stockC.index[1])[0:10],str(predic_stockC.index[-1])[0:10]), fontsize=20)
plt.legend(loc='best',bbox_to_anchor=(1, 1))
plt.xlim(xmin=stockC.index[-n-5],xmax=stockC.index[-1])
plt.xlabel('\n Date', fontsize=15)
plt.ylabel('Closing Price (₱)\n', fontsize=15) 

In [ ]:
predic_stockC.columns[0:-1].tolist()

In [ ]:
sns.color_palette()[0:-1]

In [ ]:
ax = stockC['Close'].plot(color='black', label='Actual Values', figsize=(10,5), lw=2, marker='o', markersize=8)
predic_stockC[predic_stockC.columns[0:-1].tolist()].plot(color=sns.color_palette()[0:-1], ax=ax, marker='o')

plt.title('{0} Stocks from {1} to {2}'.format(label,str(predic_stockC.index[1])[0:10],str(predic_stockC.index[-1])[0:10]), fontsize=20)
plt.legend(loc='best',bbox_to_anchor=(1, 1))
plt.xlim(xmin=stockC.index[-n-5],xmax=stockC.index[-1])
plt.xlabel('\n Date', fontsize=15)
plt.ylabel('Closing Price (₱)\n', fontsize=15) 

Plot every column in regs_stockC except MLP

In [ ]:
predic_stockC.columns[1]

In [ ]:
predic_stockC.columns[2:-1]

In [ ]:
predic_stockC.columns[2:-1]

In [ ]:
[predic_stockC.columns[0]]

In [ ]:
[predic_stockC.columns[0]]+predic_stockC.columns[2:5].tolist()

In [ ]:
[sns.color_palette()[0]]+sns.color_palette()[2:5]

In [ ]:
ax = stockC['Close'].plot(color='black', label='Actual Values', figsize=(10,5), lw=2, marker='o', markersize=8)
predic_stockC[[predic_stockC.columns[0]]+predic_stockC.columns[2:5].tolist()].plot(
    color=[sns.color_palette()[0]]+sns.color_palette()[2:5], ax=ax, marker='o')

plt.title('{0} Stocks from {1} to {2}'.format(label,str(predic_stockC.index[1])[0:n],str(predic_stockC.index[-1])[0:10]), fontsize=20)
plt.legend(loc='best',bbox_to_anchor=(1, 1))
plt.xlim(xmin=stockC.index[-n-5],xmax=stockC.index[-1])
plt.ylim(ymin=200, ymax=310)
plt.xlabel('\n Date', fontsize=15)
plt.ylabel('Closing Price (₱)\n', fontsize=15) 

## 1.9.2: Plot Lin, Ran, Dec forecasts with Plotly 

In [ ]:
pl_predic_stockC = pd.concat([stockC['Close'],predic_stockC],axis=1)
pl_predic_stockC.rename(columns={'Close': 'Actual Closing Prices'}, inplace=True)

In [ ]:
pl_predic_stockC.columns

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
import cufflinks as cf

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
cf.go_offline()

In [ ]:
[pl_predic_stockC.columns[0]]

In [ ]:
pl_predic_stockC.columns[3:-1]

In [ ]:
ideal_plots = [pl_predic_stockC.columns[0]]+pl_predic_stockC.columns[3:-1].tolist()
ideal_plots

In [ ]:
pl_predic_stockC[ideal_plots].iplot(mode='lines+markers', 
                                    title='{0} Stocks from {1} to {2}'.format(label, 
                                                                              str(pl_predic_stockC.index[-n])[0:10], 
                                                                              str(pl_predic_stockC.index[-1])[0:10]), 
                                    xTitle='\nDate', 
                                    yTitle='Closing Price (₱)\n')

# Part 1.2: Predict JFC stocks 10 days later

In [ ]:
stock.tail()

In [ ]:
pred_n=10
stock['Prediction'] = stock['Close'].shift(-pred_n)
stock.tail(pred_n+1)

In [ ]:
X = preprocessing.scale(stock.drop(['Prediction'], axis=1))

In [ ]:
X_predict = X[-pred_n:]
X = X[:-pred_n]

In [ ]:
y = stock['Prediction'][:pred_n]

In [ ]:
predictors = [SVR(), NNR(), LRR(), RFR(), DTR(), GPR()]

forecast = []
pred_cols = []
accuracy = []

fig, ax = plt.subplots(6,2, figsize=(12,24))

for pred, color, i in zip(predictors, sns.color_palette(), range(len(predictors))):
    pred.fit(X_train, y_train)
    accu = pred.score(X_test,y_test)
    predictions = pred.predict(X_test)
            
    sns.regplot(y_test,predictions, color=color, ax=ax[i,0])
    ax[i,0].set_title('{0} accuracy = {1:.2f}'.format(str(pred)[:3], (accu*100).round(2))+'%')
    ax[i,0].set_xlabel('Y Test')
    ax[i,0].set_ylabel('Predicted Y')
    
    sns.distplot((y_test-predictions), bins=50, color=color, ax=ax[i,1])
    ax[i,1].set_title('{0} Residual Histogram'.format(str(pred)[:3]))
    
    forecast.append(pred.predict(X_predict))
    pred_cols.append(str(pred)[:3])
    accuracy.append('{0} accuracy = {1:.2f}'.format(str(pred)[:3], (accu*100).round(2))+'%')
    
fig.suptitle('Performance of Scikit-learn Regressors for Closing Stock Price Prediction', fontsize=15, y=1.01)
sns.set_style('whitegrid')
fig.tight_layout()

In [ ]:
np.array(forecast).shape

In [ ]:
np.array(forecast)

In [ ]:
regs_forecast = np.array(forecast).transpose()
regs_forecast

Create datetime index for n days that excludes weekends and holidays. 

Business days only, minus holidays.

In [ ]:
last_day = stock.index[-1]
last_day

In [ ]:
last_day = stock.index[-1].to_pydatetime()
last_day

In [ ]:
holiday_dates=pd.DatetimeIndex([pd.to_datetime('2018-03-29'), pd.to_datetime('2018-03-30'), pd.to_datetime('2018-04-09')])
holiday_dates

In [ ]:
len(holiday_dates)

In [ ]:
from datetime import datetime, timedelta

In [ ]:
forecast_dates = (pd.bdate_range(last_day, periods=((pred_n+1)+len(holiday_dates)))).drop(holiday_dates)
forecast_dates

In [ ]:
accuracy

In [ ]:
regs_stock = pd.DataFrame(data=regs_forecast,
                    index=forecast_dates[1:],
                    columns=accuracy)
regs_stock

Create forecast link dataframe

In [ ]:
stock['Close'][-1:]

In [ ]:
forecast_rowlink = []
for i in regs_stock.columns:
    forecast_rowlink.append(stock['Close'][-1])
np.array([forecast_rowlink])

In [ ]:
forecast_link_stock = pd.DataFrame(data=np.array([forecast_rowlink]), 
                                   index=stock.index[-1:], 
                                   columns=accuracy)
forecast_link_stock

In [ ]:
predic_stock = pd.merge(forecast_link_stock,regs_stock, how='outer').set_index(forecast_dates)
predic_stock

In [ ]:
YouTubeVideo("VA41XQMUi-4", start=start, end=end, autoplay=1)

In [ ]:
pl_predic_stock = pd.concat([stockC['Close'],predic_stock],axis=1)
pl_predic_stock.rename(columns={'Close': 'Actual Closing Prices'}, inplace=True)
pl_predic_stock

In [ ]:
ideal_plots = [pl_predic_stock.columns[0]]+pl_predic_stock.columns[3:-1].tolist()
ideal_plots

In [ ]:
pl_predic_stock[ideal_plots].iplot(mode='lines+markers', 
                                    title='{0} Predicted Closing Prices from {1} to {2}'.format(label, 
                                                                              str(pl_predic_stock.index[-pred_n])[0:10], 
                                                                              str(pl_predic_stock.index[-1])[0:10]), 
                                    xTitle='\nDate', 
                                    yTitle='Closing Price (₱)\n')